In [27]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, concatenate
from sklearn.preprocessing import StandardScaler
from transformers import BertTokenizer, BertModel

import torch
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

## Datos de entrenamiento
En el ficher `apple_news.csv` hay un listado de noticias desde el año `efwefwe` hasta `sdfdgds`. Los datos tienen la siguiente estructura:

- _ticker_: Abreviatura del stock, en este caso **AAPL**
- _Date_: Fecha de la noticia
- _title_: Título de la noticia
- _content_: Contenido de la noticia
- _Open_: Precio de apertura de la acción ese día
- _Close_: Precio de cierre de la acción ese día
- _High_: Máximo precio alcanzado ese día
- _Low_: Mínimo precio alcanzado ese día
- _Volume_: Volumen de transacciones procesada ese día
- _label_: **0** si el precio subió o **1** si cayó o se mantuvo

Los demás datos no sabemos específicamente su significado o directamente no es relevante para nosotros.
Estos datos tienen algunos inconvenientes, por lo que tenemos que hacerle un preprocesamiento antes de continuar.

1. Las fechas están en formato `object`, por lo que hay que convertirlas a fechas para poder usarlas
2. Hay dias en donde no hay noticias y hay otros dias en donde hay mas de una noticia. Para esto lo que haremos será convinar todas las noticias del mismo día y convertirlas en una sola
3. Para los dias que no tiene noticias(ya veremos)

In [38]:
news_df = pd.read_csv('apple_news.csv')
print('Shape: ', news_df.shape)
news_df.head(3)

Shape:  (15975, 13)


,Unnamed: 0,ticker,Date,category,title,content,Open,High,Low,Close,Adj Close,Volume,label
0,0,AAPL,2020-01-27,opinion,Apple Set To Beat Q1 Earnings Estimates Tech ...,Technology giant Apple NASDAQ AAPL is set ...,77.514999,77.942497,76.220001,77.237503,75.793358,161940000,0
1,1,AAPL,2020-01-27,opinion,Tech Daily Intel Results Netflix Surge Appl...,The top stories in this digest are Intel s N...,77.514999,77.942497,76.220001,77.237503,75.793358,161940000,0
2,2,AAPL,2020-01-27,opinion,7 Monster Stock Market Predictions For The Wee...,S P 500 SPY \nThis week will be packed with e...,77.514999,77.942497,76.220001,77.237503,75.793358,161940000,0


In [46]:
news_df['Date'] = pd.to_datetime(news_df['Date'])
amount_dates = news_df['Date'].unique()

print('Información básica sobre las noticias: ')
print(f'\tCantidad de noticias: {news_df.shape[0]}')
print(f'\tCantidad de fechas con noticias: {len(amount_dates)}')
print(f'\tNoticias sobrantes: {news_df.shape[0] - len(amount_dates)}')

Información básica sobre las noticias: 
	Cantidad de noticias: 15975
	Cantidad de fechas con noticias: 1654
	Noticias sobrantes: 14321


In [47]:
news_df = news_df.sort_values(by='Date', ascending=False)
news_df = news_df.groupby('Date').agg({
    'title': ' '.join,
    'content': ' '.join,
    'Open': 'first',
    'Close': 'first',
    'High': 'first',
    'Low': 'first',
    'Volume': 'first'
}).reset_index()
# Display the combined news DataFrame
print('New Shape: ', news_df.shape)
news_df

New Shape:  (1654, 8)


,Date,title,content,Open,Close,High,Low,Volume
0,2012-07-23,Summer Heat Scorches Europe And U S Apple Ear...,Europe flares as summer heat continues Summer...,21.228571,21.565357,21.639286,20.989643,487975600
1,2012-07-24,Market Bait And Switch,That is the sound we are going to hear soon fr...,21.692142,21.461430,21.774286,21.375357,565132400
2,2012-07-27,Will AAPL Fall From The Tree,Apple s AAPL sales for the third quarter mis...,20.536072,20.898571,20.922501,20.413929,403936400
3,2012-07-30,Bulls Snatch Victory From Jaws of Defeat,Last week the bulls pulled another save out o...,21.104286,21.251072,21.408571,20.993570,379142400
4,2012-07-31,What s Driving China s Real Estate Rally Par...,In the preceding posts I examined the first t...,21.543928,21.812857,21.846430,21.525715,462327600
...,...,...,...,...,...,...,...,...
1649,2020-01-21,How to download Apple Card data into a spreads...,Apple introduced a feature on Tuesday that all...,79.297501,79.142502,79.754997,79.000000,110843200
1650,2020-01-22,Dow Jones News IBM Reports Strong Results Ap...,Some positive earnings results helped boost th...,79.644997,79.425003,79.997498,79.327499,101832400
1651,2020-01-23,Apple Boosts Chip Orders From Main Foundry Sup...,Apple s NASDAQ AAPL iPhone 11 has been selli...,79.480003,79.807503,79.889999,78.912498,104472000
1652,2020-01-24,Fiscal policies of main Irish parties vying fo...,DUBLIN Reuters Ireland s two largest parti...,80.062500,79.577499,80.832497,79.379997,146537600


Pasamos de tener `15975` noticias a solo quedarnos con `1654`

In [48]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def preprocess_news(news_list, tokenizer, max_length=512):
  input_ids = []
  attention_masks = []

  for news in tqdm(news_list):
    # Tokenización y truncamiento/padding de las noticias
    ecoded_dict = tokenizer.encode_plus(
        news, # Texto a tokenizar
        add_special_tokens=True, # Añador [CLS] y [SEP]
        max_length=max_length, # Máxima longitud
        truncation=True, # Truncamiento
        pad_to_max_length=True, # Padding
        return_attention_mask=True, # Devuelve la mascara de atención
        return_tensors='pt' # Devuelve los tensores de PyTorch
    )

    input_ids.append(ecoded_dict['input_ids'])
    attention_masks.append(ecoded_dict['attention_mask'])

  # Convertir la lista a tensores
  input_ids = torch.cat(input_ids, dim=0)
  attention_masks = torch.cat(attention_masks, dim=0)

  return input_ids, attention_masks


new_list = news_df['content'].tolist()
input_ids, attention_masks = preprocess_news(new_list, tokenizer)

  0%|          | 0/1654 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2699: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
 18%|█▊        | 290/1654 [00:08<00:40, 33.45it/s]


KeyboardInterrupt: 

In [ ]:
# Mostrar tamaños de los tensores
print(f"Input IDs shape: {input_ids.shape}")
print(f"Attention Masks shape: {attention_masks.shape}")

Input IDs shape: torch.Size([15975, 512])
Attention Masks shape: torch.Size([15975, 512])


### Paso 3: Extraer Embeddings usando BERT
Una vez que tienes las noticias tokenizadas y convertidas en tensores, puedes pasar estos tensores a un modelo BERT para extraer embeddings. Estos embeddings se pueden usar como características en tu modelo LSTM:

In [ ]:
# Crear un DataLoader para manejar el procesamiento en lotes
batch_size = 10  # Puedes ajustar el tamaño del lote según tus recursos
dataset = TensorDataset(input_ids, attention_masks)
dataloader = DataLoader(dataset, batch_size=batch_size)

model = BertModel.from_pretrained('bert-base-uncased')
model.eval()

# Función para obtener embeddings de BERT en lotes
def get_bert_embeddings(dataloader, model):
    all_embeddings = []

    for batch in dataloader:
        batch_input_ids, batch_attention_masks = batch

        with torch.no_grad():
            outputs = model(batch_input_ids, attention_mask=batch_attention_masks)
            batch_embeddings = outputs.last_hidden_state

        all_embeddings.append(batch_embeddings)

    return torch.cat(all_embeddings, dim=0)

# Obtener los embeddings en lotes
embeddings = get_bert_embeddings(dataloader, model)

# Mostrar tamaño de los embeddings
print(f"Embeddings shape: {embeddings.shape}")

KeyboardInterrupt: 

### Paso 4: Integración con el Modelo LSTM
Ahora que tienes los embeddings de las noticias, puedes integrarlos en tu modelo LSTM tal como se describió en el primer ejemplo. Aquí hay una versión modificada del paso 4 para integrar estos embeddings en tu modelo LSTM:

In [ ]:
prices = pd.read_csv('apple_stock_prices.csv')
prices['Date'] = pd.to_datetime(prices['Date'])
prices.set_index('Date', inplace=True)

In [ ]:
scaler = StandardScaler()
prices_scaled = scaler.fit_transform(prices[prices[['Close']]])

In [ ]:
combined_data = prices.join(news, how='inner')

sequence_length = 10
X_prices = []
X_news = []
y = []

for u in range(len(combined_data) - sequence_length):
    X_prices.append(prices_scaled[u:u+sequence_length])
    # Asumimos que 'news_vectors' ya está preprocesado y sincronizado con 'prices'
    X_news.append(news_vectors[u:u+sequence_length])
    y.append(prices_scaled[u+sequence_length])

X_prices = np.array(X_prices)
X_news = np.array(X_news)
y = np.array(y)

In [ ]:
# Definición del modelo
price_input = Input(shape=(sequence_length, 1), name='price_input')
news_input = Input(shape=(sequence_length, embeddings.shape[-1]), name='news_input')

price_lstm = LSTM(50, return_sequences=False)(price_input)
news_lstm = LSTM(50, return_sequences=False)(news_input)

combined = concatenate([price_lstm, news_lstm])
dense = Dense(50, activation='relu')(combined)
output = Dense(1, activation='linear')(dense)

model = Model(inputs=[price_input, news_input], outputs=output)

# Compilación del modelo
model.compile(optimizer='adam', loss='mean_squared_error')

# Entrenamiento del modelo
model.fit([X_prices, X_news], y, epochs=20, batch_size=32, validation_split=0.2)

# # Evaluación del modelo
# # Asumiendo que tienes un conjunto de test
# X_prices_test = ...
# X_news_test = ...
# y_test = ...

model.evaluate([X_prices_test, X_news_test], y_test)